## Different Anchors (explains sys_composer.morph_system)

In [3]:
import x_xy

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

import mediapy as media

def show_video(sys, xs: x_xy.Transform) -> None:
    assert sys.dt == 0.01
    # only render every fourth to get a framerate of 25 fps
    frames = x_xy.render(sys, [xs[i] for i in range(0, xs.shape(), 4)], camera="targetfar", height=480, width=640)
    # convert rgba to rgb
    frames = [frame[..., :3] for frame in frames]
    media.show_video(frames, fps=25)

In this system the middle segment `seg2` acts as "anchor".

In [4]:
xml_str = """
<x_xy model="three_segment_kinematic_chain">
    <options gravity="0 0 9.81" dt="0.01"/>
    <worldbody>
        <body name="seg2" joint="free" pos="0 0 1">
            <geom type="box" mass="0.1" pos="0.5 0 0" dim="1 0.25 0.2" color="self"/>
            <body name="seg1" joint="ry">
                <geom type="box" mass="0.1" pos="-0.5 0 0" dim="1 0.25 0.2" color="self"/>
                <body name="imu1" joint="frozen" pos="-0.5 0 0.125">
                    <geom type="box" mass="0.05" dim="0.2 0.2 0.05" color="orange"/>
                </body>
            </body>
            <body name="seg3" joint="rz" pos="1 0 0">
                <geom type="box" mass="0.1" pos="0.5 0 0" dim="1 0.25 0.2" color="self"/>
                <body name="imu2" joint="frozen" pos="0.5 0 -0.125">
                    <geom type="box" mass="0.05" dim="0.2 0.2 0.05" color="orange"/>
                </body>
            </body>
        </body>
    </worldbody>
</x_xy>
"""

sys = x_xy.load_sys_from_str(xml_str)

gen = x_xy.build_generator(sys, x_xy.RCMG_Config(T=10.0, t_max=1.5, dang_max_free_spherical=0.1, dpos_max=0.1))
_, xs = gen(jax.random.PRNGKey(1))
show_video(sys, xs)


Rendering frames..: 100%|██████████| 250/250 [00:01<00:00, 175.10it/s]


This browser does not support the video tag.


Can you see what i mean? The middle segment has all the "global rotation and translation".

Let's move the anchor to `seg1` but without changing the xml syntax. This can be done with the subpackage `sys_composer`.

In [5]:
from x_xy.subpkgs import sys_composer

In [6]:
# the new parents of seg2, seg1, imu1, seg3, imu2 are ...
new_parents = ["seg1", -1, "seg1", "seg2", "seg3"]
sys = sys_composer.morph_system(sys, new_parents=new_parents)

gen = x_xy.build_generator(sys, x_xy.RCMG_Config(T=10.0, t_max=1.5, dang_max_free_spherical=0.1, dpos_max=0.1))
_, xs = gen(jax.random.PRNGKey(1))
show_video(sys, xs)


Rendering frames..: 100%|██████████| 250/250 [00:01<00:00, 182.39it/s]


This browser does not support the video tag.


Pretty cool, ha? :)